<a href="https://colab.research.google.com/github/HILALOZTEMEL/BernoulliRBM-Chi_Scores-AutoEncoder-with--NSL_KDD-data-set/blob/main/BernoulliRBM_Chi_Scores_AutoEncoder_with_NSL_KDD_data_set_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.neural_network import BernoulliRBM
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn import linear_model


#kimlik doğrulama işlemi :drive in içinde çalışma ortamı sağlarız serverlarda
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

# Loading data from train.csv file
test_df = pd.read_csv("/gdrive/My Drive/İstanbul Gedik Üniversitesi/DataMining/nsl-kdd/NSL_KDD_Test.csv",header=None, names = col_names)
train_df = pd.read_csv("/gdrive/My Drive/İstanbul Gedik Üniversitesi/DataMining/nsl-kdd/NSL_KDD_Train.csv", header=None, names = col_names)


print('Dimensions of the Training set:',train_df.shape)
print('Dimensions of the Test set:',test_df.shape)

Mounted at /gdrive
/gdrive
Dimensions of the Training set: (125973, 42)
Dimensions of the Test set: (22544, 42)


In [ ]:
train_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [ ]:
test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,private,REJ,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
1,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_df['protocol_type'] = le.fit_transform(train_df['protocol_type'])
test_df['protocol_type'] = le.transform(test_df['protocol_type'])
train_df['service'] = le.fit_transform(train_df['service'])
test_df['service'] =le.transform(test_df['service'])
train_df['flag'] = le.fit_transform(train_df['flag'])
test_df['flag'] = le.transform(test_df['flag'])

In [ ]:
train_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,20,9,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,2,44,9,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,1,49,5,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
3,0,1,24,9,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,1,24,9,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [ ]:
test_df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,1,49,1,0,0,0,0,0,0,...,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
1,0,1,49,1,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune
2,2,1,20,9,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal
3,0,0,14,9,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint
4,1,1,60,2,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan


In [ ]:
X_train = train_df.drop([ "label"], axis=1)
y_train = train_df["label"]

X_test = test_df.drop([ "label"], axis=1)
y_test = test_df["label"]

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((125973, 41), (22544, 41), (125973,), (22544,))

In [ ]:
rbm = BernoulliRBM(n_components=40, learning_rate=0.06, n_iter=100, verbose=1, random_state=101)
rbm.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 1, pseudo-likelihood = -870955488.82, time = 1.84s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 2, pseudo-likelihood = -1741911772.46, time = 2.34s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


...

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 99, pseudo-likelihood = -86224670966.80, time = 2.22s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but BernoulliRBM was fitted with feature names
  "X does not have valid feature names, but"


[BernoulliRBM] Iteration 100, pseudo-likelihood = -87095627247.36, time = 2.22s


BernoulliRBM(learning_rate=0.06, n_components=40, n_iter=100, random_state=101,
             verbose=1)

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((125973, 41), (22544, 41), (125973,), (22544,))

In [ ]:
from sklearn.feature_selection import chi2

chi_scores = chi2(X_train,y_train)
chi_scores

(array([6.66222035e+08, 8.36458475e+03, 2.09526973e+05, 7.92604404e+04,
        1.48197070e+11, 8.99826760e+10, 9.06792264e+04, 3.55373059e+05,
        1.10219991e+04, 3.78381716e+05, 4.84377513e+04, 4.08918661e+04,
        3.19877755e+04, 1.47764189e+04, 5.70428355e+02, 3.70034612e+04,
        7.61510443e+03, 1.59089095e+04, 1.94840687e+03,            nan,
        8.70617585e-01, 1.13165561e+04, 1.10590608e+07, 1.22404857e+07,
        6.47714154e+04, 6.55820198e+04, 2.66173733e+04, 2.69290569e+04,
        3.22479288e+04, 2.32778581e+04, 2.37542578e+04, 2.47169802e+06,
        7.74459596e+06, 3.24617982e+04, 2.38525192e+04, 3.93504338e+04,
        3.10160105e+04, 6.46178726e+04, 6.69249553e+04, 1.91714534e+04,
        2.67982444e+04]),
 array([0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000, 0.0000000e+000, 0.0000000e+000,
        0.0000000e+000, 0.0000000e+000

In [ ]:
X_train.shape,X_test.shape , y_train.shape ,y_test.shape

((125973, 41), (22544, 41), (125973,), (22544,))

In [ ]:
#Importing the Libraries
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model


In [ ]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.



# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

# AutoEncoder Model Preparation
n_inputs = X_train.shape[1]
# define encoder
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)


In [ ]:
# output layer
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# fit the autoencoder model to reconstruct input
history = model.fit(X_train, X_train,shuffle=True, epochs=20, batch_size=20, validation_data=(X_test, X_test))


Epoch 1/20
6299/6299 [==============================] - 24s 4ms/step - loss: 54.8315 - accuracy: 0.0356 - val_loss: 55.4461 - val_accuracy: 0.0091
Epoch 2/20
6299/6299 [==============================] - 23s 4ms/step - loss: 55.2440 - accuracy: 0.0291 - val_loss: 52.5907 - val_accuracy: 0.0472
Epoch 3/20
6299/6299 [==============================] - 22s 4ms/step - loss: 54.5651 - accuracy: 0.1097 - val_loss: 60.9697 - val_accuracy: 0.3844
Epoch 4/20
6299/6299 [==============================] - 22s 3ms/step - loss: 54.3815 - accuracy: 0.2308 - val_loss: 48.8831 - val_accuracy: 0.0158
Epoch 5/20
6299/6299 [==============================] - 22s 3ms/step - loss: 54.9482 - accuracy: 0.0330 - val_loss: 56.7247 - val_accuracy: 0.1781
Epoch 6/20
6299/6299 [==============================] - 22s 3ms/step - loss: 52.2091 - accuracy: 0.0124 - val_loss: 51.3491 - val_accuracy: 0.0117
Epoch 7/20
6299/6299 [==============================] - 23s 4ms/step - loss: 56.2371 - accuracy: 0.0222 - val_loss: 49

In [ ]:
#Using Logistic Regression Algorithm to the Training Set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=0)

In [ ]:
# Returns a NumPy Array
# Predict for One Observation (image)
classifier.predict(X_test[0].reshape(1,-1))

array(['neptune'], dtype=object)

In [ ]:
X_test.shape

(22544, 41)

In [ ]:
classifier.predict(X_test[0:10])

array(['neptune', 'neptune', 'normal', 'nmap', 'normal', 'normal',
       'normal', 'normal', 'normal', 'normal'], dtype=object)

In [ ]:
predictions = classifier.predict(X_test)

In [ ]:
# Use score method to get accuracy of model
score = classifier.score(X_test, y_test)
print(score)

0.7048438608942512
